In [58]:
using ForwardDiff
using Plots
using Optim

In [68]:
function norm(a)
    sqrt.(sum(a.^2))
end

norm (generic function with 1 method)

In [85]:
function horner(a,x)
    r = 0.0
    for i = length(a):-1:2
       r = x*(a[i]+r)
    end
    return a[1]+r
end

horner (generic function with 1 method)

In [108]:
xdata = collect(range(-10,10,10000))
a = [-10,1,2]
a0 = zeros(3)
ydata = (x->horner(a,x)).(xdata);

In [109]:
E(a) = sum(( ((xx->horner(a,xx)).(xdata)) - ydata).^2)
∇E(a) = ForwardDiff.gradient(E,a)

∇E (generic function with 1 method)

In [120]:
function grad_descent(lr,a0,it=10000)
    a = a0
    i = 0
    while i<it
        grad = ∇E(a)
        grad /= norm(grad)
        a -= lr*grad
        i += 1
    end
    a
end

grad_descent (generic function with 2 methods)

In [122]:
grad_descent(0.1,a0)

3-element Vector{Float64}:
 -9.881805283858665
  0.9999999999999983
  1.9480280088393882